<a href="https://colab.research.google.com/github/LogeshChandran/Huggingface_finetune/blob/main/upload_dataset_into_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## newssroom data set upload to hugging face

---

In [2]:
# Install Git LFS
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
! git lfs install

Git LFS initialized.


In [4]:
! git clone https://huggingface.co/datasets/Logeshkc/newsroom

Cloning into 'newsroom'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 39 (delta 11), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (39/39), 10.27 KiB | 809.00 KiB/s, done.
Filtering content: 100% (7/7), 3.01 GiB | 47.64 MiB/s, done.


In [20]:
! ls

newsroom  sample_data


In [19]:
! mkdir newsroom/dataset

In [ ]:
! pip install datasets

In [11]:
import pandas as pd
import json,gzip


In [ ]:
dir()

In [ ]:
# data = []
# names = ["train","test","dev"]
# # names = ["test","dev"]
# for name in names:
#     path = f"newsroom/data/{name}.jsonl.gz"
#     data = []
#     with gzip.open(path) as f:
#         for ln in f:
#             obj = json.loads(ln)
#             data.append(obj)
#     data_df = pd.DataFrame(data)
#     del data
#     data_df.to_json(f"newsroom/dataset/{name}.json")
#     del data_df

In [2]:
# First, install dask
!pip install dask


In [ ]:
import gzip
import json
import pandas as pd
from datasets import Dataset, DatasetDict

# Processing in chunks
def process_data_in_chunks(name, chunk_size=10000):
    path = f"newsroom/data/{name}.jsonl.gz"
    data = []
    chunk_count = 0
    with gzip.open(path) as f:
        for ln in f:
            obj = json.loads(ln)
            data.append(obj)

            # When the chunk reaches the specified size, save it to disk
            if len(data) >= chunk_size:
                data_df = pd.DataFrame(data)
                print(f"newsroom/dataset/{name}_chunk_{chunk_count}.json")
                data_df.to_json(f"newsroom/dataset/{name}_chunk_{chunk_count}.json")
                chunk_count += 1
                data.clear()  # Clear memory by clearing the list

        # Save the last chunk if there are remaining records
        if len(data) > 0:
            data_df = pd.DataFrame(data)
            print(f"newsroom/dataset/{name}_chunk_{chunk_count}.json")
            data_df.to_json(f"newsroom/dataset/{name}_chunk_{chunk_count}.json")
            data.clear()

# Process each dataset in chunks
for name in ["train", "test", "dev"]:
    process_data_in_chunks(name)


In [6]:
from datasets import Dataset, DatasetDict, concatenate_datasets # Import the concatenate_datasets function

In [ ]:
# Read and combine the chunks for each dataset
def load_chunks(name):
    chunks = []
    chunk_index = 0
    while True:
        try:
            # Read each chunk into a DataFrame and convert to Dataset
            print(f"newsroom/dataset/{name}_chunk_{chunk_index}.json")
            df = pd.read_json(f"newsroom/dataset/{name}_chunk_{chunk_index}.json")
            dataset_chunk = Dataset.from_pandas(df)
            chunks.append(dataset_chunk)
            chunk_index += 1
        except FileNotFoundError:
            # If no more chunks are found, break the loop
            break
    return concatenate_datasets(chunks)  # Use concatenate_datasets to combine all the chunks

# Convert chunked datasets
train_dataset = load_chunks("train")
test_dataset = load_chunks("test")
dev_dataset = load_chunks("dev")

# Create the DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": dev_dataset
})

In [32]:
del train_dataset
del test_dataset
del dev_dataset

In [31]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['url', 'archive', 'title', 'date', 'text', 'summary', 'compression', 'coverage', 'density', 'compression_bin', 'coverage_bin', 'density_bin', '__index_level_0__'],
        num_rows: 995041
    })
    test: Dataset({
        features: ['url', 'archive', 'title', 'date', 'text', 'summary', 'compression', 'coverage', 'density', 'compression_bin', 'coverage_bin', 'density_bin', '__index_level_0__'],
        num_rows: 108862
    })
    validation: Dataset({
        features: ['url', 'archive', 'title', 'date', 'text', 'summary', 'compression', 'coverage', 'density', 'compression_bin', 'coverage_bin', 'density_bin', '__index_level_0__'],
        num_rows: 108837
    })
})

In [13]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [34]:
# Push to Hugging Face Hub
repo_name = "Logeshkc/newsroom"
dataset_dict.push_to_hub(repo_name)

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/109 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Logeshkc/newsroom/commit/408396a3ac03812b924918031daf564e507e2365', commit_message='Upload dataset', commit_description='', oid='408396a3ac03812b924918031daf564e507e2365', pr_url=None, pr_revision=None, pr_num=None)

In [61]:
! huggingface-cli whoami

Logeshkc
